In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az



In [5]:
# Load your dataset
data = pd.read_csv('/media/momty/Extreme SSD/svc/2023/2023_prospect_params_GT (copy).csv')

# Ensure the dataset includes 'CHL', 'CAR', 'ANT', 'CBC', 'Nstruct' and any other parameters for conditioning


In [6]:
import pymc as pm
import numpy as np
import pandas as pd

def bayesian_regression(data, predictors, target='CHL'):
    with pm.Model() as model:
        # Priors for unknown model parameters
        intercept = pm.Normal('Intercept', mu=0, sigma=10)
        slopes = pm.Normal('slopes', mu=0, sigma=10, shape=len(predictors))

        # Convert predictors data to a shared variable
        predictors_data = pm.Data('predictors_data', data[predictors].values)

        # Constructing the linear model expression
        mu = intercept + pm.math.dot(predictors_data, slopes)

        # Standard deviation is a positive parameter; use HalfNormal
        sigma = pm.HalfNormal('sigma', sigma=1)

        # Likelihood (sampling distribution) of observations
        y_obs_data = pm.Data('y_obs_data', data[target].values)
        y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y_obs_data)

        # MCMC sampling
        trace = pm.sample(1000, return_inferencedata=True, target_accept=0.95)

    return model, trace


In [7]:
def make_predictions(model, trace, new_predictors, new_target):
    with model:
        # Update the model's observed data
        pm.set_data({'predictors_data': new_predictors, 'y_obs_data': new_target})

        # Generate posterior predictive samples
        post_pred = pm.sample_posterior_predictive(trace)
        
        # Correctly accessing the predictions
        predictions = post_pred.posterior_predictive['y_obs'].values

        return predictions




In [8]:
# Fit the model with existing data
model, trace = bayesian_regression(data, predictors=['CAR', 'ANT', 'CBC', 'Nstruct'])

# Example new data for prediction
new_predictors = np.array([[7.6, 4.18, 0.004, 2.22]])  # Real values to be used
new_target = np.array([0])  # Dummy target value

# Make predictions with new data
predictions = make_predictions(model, trace, new_predictors, new_target)

# Calculate prediction statistics
predicted_mean = np.mean(predictions)
prediction_intervals = np.percentile(predictions, [5, 95])

print("Predicted Mean CHL:", predicted_mean)
print("Prediction Intervals for CHL:", prediction_intervals)



Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, slopes, sigma]


/home/momty/miniconda3/envs/pymc_env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 15 seconds.
Sampling: [y_obs]


/home/momty/miniconda3/envs/pymc_env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Predicted Mean CHL: 37.76100348275859
Prediction Intervals for CHL: [31.39314042 44.19190611]
